<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/deepseek_coin_tarama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time
import numpy as np  # Eksik olan kütüphane eklendi

# CoinGecko API erişim bilgileri
COINGECKO_URL = "https://api.coingecko.com/api/v3/coins/markets"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# CoinGecko'dan veri çekme fonksiyonu
def get_coingecko_data(coin_ids, vs_currency='usd'):
    try:
        params = {
            'vs_currency': vs_currency,
            'ids': ','.join(coin_ids),
            'order': 'market_cap_desc',
            'per_page': 100,
            'page': 1,
            'sparkline': False
        }
        response = requests.get(COINGECKO_URL, headers=HEADERS, params=params, timeout=15)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"CoinGecko API Hatası: {str(e)}")
        return None

# Veriyi işleme fonksiyonu
def process_coingecko_data(data):
    if not data:
        return pd.DataFrame()

    df = pd.DataFrame(data)
    df = df[['id', 'symbol', 'current_price', 'high_24h', 'low_24h', 'price_change_percentage_24h']]
    df.rename(columns={
        'id': 'Coin',
        'symbol': 'Sembol',
        'current_price': 'Fiyat',
        'high_24h': '24s Yüksek',
        'low_24h': '24s Düşük',
        'price_change_percentage_24h': '24s Değişim (%)'
    }, inplace=True)
    return df

# Sinyal analizi fonksiyonu
def analyze_data(df):
    if df.empty:
        return df

    # Örnek sinyal: Fiyat 24 saat içinde %5'ten fazla artmışsa "AL"
    df['Sinyal'] = np.where(df['24s Değişim (%)'] > 5, 'AL', 'SAT')
    return df

# Ana tarama fonksiyonu
def scan_coins(coin_ids):
    data = get_coingecko_data(coin_ids)
    if data:
        df = process_coingecko_data(data)
        if not df.empty:
            df = analyze_data(df)
            return df
    return pd.DataFrame()

# Çalıştırma
if __name__ == "__main__":
    coin_ids = ['bitcoin', 'ethereum', 'binancecoin', 'cardano', 'solana']  # CoinGecko ID'leri
    while True:
        print(f"\n--- Tarama Zamanı: {time.ctime()} ---")
        result = scan_coins(coin_ids)
        if not result.empty:
            print(result.to_string(index=False))
        else:
            print("Veri alınamadı.")
        time.sleep(300)  # 5 dakikada bir tarama


--- Tarama Zamanı: Mon Feb  3 18:19:33 2025 ---
       Coin Sembol        Fiyat   24s Yüksek    24s Düşük  24s Değişim (%) Sinyal
    bitcoin    btc 99265.000000 99605.000000 92460.000000          1.92122    SAT
   ethereum    eth  2712.290000  2980.090000  2368.280000         -8.80534    SAT
     solana    sol   206.400000   209.520000   183.180000          1.66380    SAT
binancecoin    bnb   595.600000   628.790000   523.650000         -4.28021    SAT
    cardano    ada     0.771886     0.823783     0.573037         -4.55132    SAT
